In [16]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.style.use('classic')

In [17]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

In [18]:
import os
import cv2
from PIL import Image
import numpy as np

In [19]:
image_directory = '299aug-nor-macular1000/'
SIZE = 150
dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
label = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.

In [20]:
macular_degeneration_images = os.listdir(image_directory + 'macular-degeneration/')
for i, image_name in enumerate(macular_degeneration_images):    #Remember enumerate method adds a counter and returns the enumerate object
    
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'macular-degeneration/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

In [21]:
normal_images = os.listdir(image_directory + 'normal/')
for i, image_name in enumerate(normal_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'normal/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset = np.array(dataset)
label = np.array(label)

In [22]:
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical

In [23]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.10, random_state = 0)

In [24]:
#Without scaling (normalize) the training may not converge. 
#Normalization is a rescaling of the data from the original range 
#so that all values are within the range of 0 and 1.
from tensorflow.keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

#Do not do one-hot encoding as it generates a shape of (num, 2)
#But the network expects an input of (num, 1) for the last layer for binary classification
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [25]:
###2 conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(1))
model.add(Activation('sigmoid'))  
#Do not use softmax for binary classification
#Softmax is useful for mutually exclusive classes, either cat or dog but not both.
#Also, softmax outputs all add to 1. So good for multi class problems where each
#class is given a probability and all add to 1. Highest one wins. 

In [26]:
#Sigmoid outputs probability. Can be used for non-mutually exclusive problems.
#But, also good for binary mutually exclusive (cat or not cat). 

model.compile(loss='binary_crossentropy',
              optimizer='adam',             #also try adam
              metrics=['accuracy'])


In [27]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)       

In [28]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         epochs = 25,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/25
113/113 [==============================] - 20s 169ms/step - loss: 0.6415 - accuracy: 0.6211 - val_loss: 0.4932 - val_accuracy: 0.7850
Epoch 2/25
113/113 [==============================] - 19s 167ms/step - loss: 0.4616 - accuracy: 0.7878 - val_loss: 0.4289 - val_accuracy: 0.8050
Epoch 3/25
113/113 [==============================] - 19s 167ms/step - loss: 0.4550 - accuracy: 0.7778 - val_loss: 0.4158 - val_accuracy: 0.7950
Epoch 4/25
113/113 [==============================] - 19s 172ms/step - loss: 0.4332 - accuracy: 0.7944 - val_loss: 0.4254 - val_accuracy: 0.7950
Epoch 5/25
113/113 [==============================] - 21s 182ms/step - loss: 0.4361 - accuracy: 0.7883 - val_loss: 0.4578 - val_accuracy: 0.7900
Epoch 6/25
113/113 [==============================] - 21s 182ms/step - loss: 0.4533 - accuracy: 0.7783 - val_loss: 0.4192 - val_accuracy: 0.8050
Epoch 7/25
113/113 [==============================] - 21s 182ms/step - loss: 0.4205 - accuracy: 0.8033 - val_loss: 0.3864 - val_ac

In [32]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         steps_per_epoch = 163,
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         validation_steps = 624,
                         shuffle = False
                     )


Epoch 1/10
163/163 [==============================] - 27s 166ms/step - loss: 0.2257 - accuracy: 0.9077 - val_loss: 0.3522 - val_accuracy: 0.8850
Epoch 2/10
163/163 [==============================] - 27s 164ms/step - loss: 0.1445 - accuracy: 0.9450 - val_loss: 0.4124 - val_accuracy: 0.8550
Epoch 3/10
163/163 [==============================] - 27s 166ms/step - loss: 0.1261 - accuracy: 0.9498 - val_loss: 0.2987 - val_accuracy: 0.8800
Epoch 4/10
163/163 [==============================] - 27s 169ms/step - loss: 0.1651 - accuracy: 0.9392 - val_loss: 0.3574 - val_accuracy: 0.8550
Epoch 5/10
163/163 [==============================] - 29s 179ms/step - loss: 0.1109 - accuracy: 0.9514 - val_loss: 0.4811 - val_accuracy: 0.8800
Epoch 6/10
163/163 [==============================] - 29s 181ms/step - loss: 0.0723 - accuracy: 0.9738 - val_loss: 0.4363 - val_accuracy: 0.8850
Epoch 7/10
163/163 [==============================] - 28s 173ms/step - loss: 0.0937 - accuracy: 0.9681 - val_loss: 0.4498 - val_ac

In [35]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         steps_per_epoch = 100,
                         epochs = 1,      
                         validation_data=(X_test,y_test),
                         shuffle = True
                     )


100/100 [==============================] - 17s 167ms/step - loss: 0.0836 - accuracy: 0.9694 - val_loss: 0.4442 - val_accuracy: 0.8150
